In [1]:
all = [var for var in globals() if var[0] != '_']
for var in all:
    del globals()[var]
del all, var

# Implementation

Author: Jørgen Schartum Dokken

This implementation is an adaptation of the work in {cite}`fundamentals-FenicsTutorial` to DOLFINx.

In this section, you will learn:
- How to use the built-in meshes in DOLFINx
- How to create a spatially varying Dirichlet boundary conditions on the whole domain boundary
- How to define a weak formulation of your PDE
- How to solve the resulting system of linear equations
- How to visualize the solution using a variety of tools
- How to compute the $L^2(\Omega)$ error and the error at mesh vertices

## Interactive tutorials
```{admonition} Run the tutorial as Jupyter notebook in browser
As this book has been published as a Jupyter Book, each code can be run in your browser as a Jupyter notebook.
To start such a notebook click the rocket symbol in the top right corner of the relevant tutorial.
```

The Poisson problem has so far featured a general domain $\Omega$ and general functions $u_D$ for
the boundary conditions and $f$ for the right hand side.
Therefore, we need to make specific choices of $\Omega, u_D$ and $f$.
A wise choice is to construct a problem  with a known analytical solution,
so that we can check that the computed solution is correct.
The primary candidates are lower-order polynomials.
The continuous Galerkin finite element spaces of degree $r$ will exactly reproduce polynomials of degree $r$.
<!-- Particularly, piecewise linear continuous Galerkin finite elements are able to exactly reproduce a quadratic polynomial on
a uniformly partitioned mesh. -->
 We use this fact to construct a quadratic function in $2D$. In particular we choose

$$
\begin{align}
 u_e(x,y)=1+x^2+2y^2
 \end{align}
$$

Inserting $u_e$ in the original boundary problem, we find that

$$
\begin{align}
    f(x,y)= -6,\qquad u_D(x,y)=u_e(x,y)=1+x^2+2y^2,
\end{align}
$$

regardless of the shape of the domain as long as we prescribe
$u_e$ on the boundary.

For simplicity, we choose the domain to be a unit square $\Omega=[0,1]\times [0,1]$  

--> 재밌네. 그니까 이 사람이 위에서 설명한게 뭐냐면.. 우리는 지금 Poisson eqn에 대해 FEM으로 풀려고 하고 있잖아. 그럼 어찌됐든 FEM을 써서 Poisson eqn을 풀 텐데, 풀어낸 결과가 원래의 Poisson eqn의 진짜 soln과 일치하는지를 검증하는 게 필요하잖아. 이 검증을 위한 가장 편한 방법이 뭐냐면, 아싸리 그냥 우리가 Poisson eqn의 해를 우리 맘대로 specific하게 미리 선정해버리는 거임. 가령 위에처럼
$$
\begin{align}
 u_e(x,y)=1+x^2+2y^2
 \end{align}
$$
과 같이 soln을 아예 미리 우리가 정해버리고(u의 하첨자 e는 exact solution을 의미) 얘를 주어진 Poisson eqn
$$
\begin{align}
 -\frac{\partial^2 u}{\partial x_1^2} - \frac{\partial^2 u}{\partial x_2^2} = f(x_1, x_2)
 \end{align}
$$
에 대입시키면 $f(x_1, y_2) = -6$이 되고 boundary에서의 Dirichlet B.C value들은 그냥 우리가 기설정한 $u(x_1, x_2)$ 식에다가 해당 boundary의 위치만 갖다넣으면 나오는 거니까, 우리가 FEM으로 풀 문제에 대한 input 정보들(weak form of PDE, source term, B.C)이 우리가 선정한 soln 식으로부터 줄줄이 다 나오는 거지. 그러면 이제 FEniCS에다가 이 정보들만 갖다가 넣어주면 되고, FEniCS로 이걸 풀면 그 결과를 우리가 기선정한 soln과 비교하면 되는 거임. 이런 방식을 "the method of manufactured solutions"라고 한다네.

This simple but very powerful method for constructing test problems is called _the method of manufactured solutions_.
First pick a simple expression for the exact solution, plug into
the equation to obtain the right-hand side (source term $f$).
Then solve the equation with this right hand side, and using the exact solution as boundary condition.
Finally, we create a program that tries to reproduce the exact solution.

Note that in many cases, it can be hard to determine if the program works if it produces an error of size
$10^{-5}$ on a $20 \times 20$ grid.
However, since we are using Sobolev spaces, we usually know about the numerical errors _asymptotic properties_.
For instance that it is proportional to $h^2$ if $h$ is the size of a cell in the mesh.
We can then compare the error on meshes with different $h$-values to see if the asymptotic behavior is correct.
This technique will be explained in detail in the chapter [Improving your fenics code](./../chapter4/convergence).  

--> 이게 뭔 소리냐면.. FEM의 오차는 Sobolev space의 이론에 따라 이미 다음과 같이 알려져 있음. $O(h^2)$ ; $h : element$  $size$ 즉 요소 사이즈의 제곱에 비례함. 요소 크기가 작으면 작을수록 오차도 제곱승으로 작아지는거임. 따라서 우리는 같은 문제를 element 사이즈를 줄여가며 재현할수록 error가 element 사이즈의 제곱에 점근적으로 비례해가는 asymptotic properties를 관찰할 수 있게 될 거임.


However, in cases where we have a solution we know that should have no approximation error,
we know that the solution should be produced to machine precision by the program.  

--> 이게 뭔 소리냐면.. 우리가 앞서서 우리 편할대로 우리가 풀 Poisson eqn에 대한 solution을 $u_e = 1 + x^2 + 2y^2$으로 골랐잖아. 이게 지금 보면 형태가 2차 다항식이잖아. 그런데 만약 우리가 이 문제에 대한 FEM에서 shape function을 2차 함수로 선정하게 되면, FEM의 solution(apporx)이 저 solution(true)을 정확하게 재현할 수 있게 될 거잖아(2차 함수로 2차 함수를 근사하는 거니까)? 이말인즉슨 FEM의 결과가 실제 이론적 해와 일치하게 된다는 거임. 단 이 때 우리는 FEM을 컴퓨터 가지고 푸니까 FEM의 해와 실제 해 간에 오차가 아예 안 존재하지는 않고 machine precision만큼의 오차는 존재하게 됨(가능한 최소의 오차만 존재한단 얘기지). 

A major difference between a traditional FEniCS code and a FEniCSx code,
is that one is not advised to use the wildcard import.
We will see this throughout this first example.

## Generating  simple meshes
The next step is to define the discrete domain, _the mesh_.
We do this by importing one of the built-in mesh generators.
We will build a {py:func}`unit square mesh<dolfinx.mesh.create_unit_square>`, i.e. a mesh spanning $[0,1]\times[0,1]$.
It can consist of either triangles or quadrilaterals.

In [2]:
from mpi4py import MPI # 병렬 연산을 위한 모듈 import
from dolfinx import mesh # mesh 짜기 위한 모듈 import
import numpy

### 1 . Domain 정의 및 mesh 만들기
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)
# MPI.COMM_WORLD 사용함으로써 우리가 사용할 여러 개의 프로세서들에 mesh 데이터를 분산시킴 ; 여기서 '프로세서'란 물리적 CPU가 아니라 MPI에서 말하는 "OS가 관리하는 독립된 프로세스(소프트웨어)"를 의미함.
# 각 프로세서는 할당된 메모리를 가지며, 프로세스 하나가 하나의 독립된 계산 "노드"로 인식됨.
# 가로 길이 1, 세로 길이 1의 2D domain을 x방향 8개, y 방향 8개로 정사각형 element를 늘어뜨려 만듦.
# Cell 타입은 quadrilateral ... FEniCS에서 element는 cell이라고 부름
# 이렇게 만들어진 mesh를 domain이라는 객체에 저장

In [ ]:
# 1-1. 생성된 mesh(domain) 객체에 어떤 정보들이 들어있는지 봐보자.
print(f'Cell(element) Type : {domain.topology.cell_type}')
print(f'Entity Types : {domain.topology.entity_types}')
print(f'Original Cell Index : {domain.topology.original_cell_index}')
print(f'domain.geometry.cmap : {domain.geometry.cmap}')
print(f'domain.geometry.dim : {domain.geometry.dim}')
print(f'domain.geometry.dofmap : {domain.geometry.dofmap}')
print(f'domain.geometry.input_global_indices : {domain.geometry.input_global_indices}')
print(f'domain.geometry.x : {domain.geometry.x}')
print(f'domain.geometry.index_map() : {domain.geometry.index_map()}')
print(f'domain.geometry.__class__ : {domain.geometry.__class__}')

Cell(element) Type : CellType.quadrilateral
Entity Types : [[CellType.point], [CellType.interval], [CellType.quadrilateral]]
Original Cell Index : [ 0  1  8  2  9 16  3 10 17 24  4 11 18 25 32  5 12 19 26 33 40  6 13 20
 27 34 41 48  7 14 21 28 35 42 49 56 15 22 29 36 43 50 57 23 30 37 44 51
 58 31 38 45 52 59 39 46 53 60 47 54 61 55 62 63]
domain.geometry.cmap : <dolfinx.fem.element.CoordinateElement object at 0x726f507efb60>
domain.geometry.dim : 2
domain.geometry.dofmap : [[ 0  1  2  3]
 [ 1  4  3  5]
 [ 2  3  6  7]
 [ 4  8  5  9]
 [ 3  5  7 10]
 [ 6  7 11 12]
 [ 8 13  9 14]
 [ 5  9 10 15]
 [ 7 10 12 16]
 [11 12 17 18]
 [13 19 14 20]
 [ 9 14 15 21]
 [10 15 16 22]
 [12 16 18 23]
 [17 18 24 25]
 [19 26 20 27]
 [14 20 21 28]
 [15 21 22 29]
 [16 22 23 30]
 [18 23 25 31]
 [24 25 32 33]
 [26 34 27 35]
 [20 27 28 36]
 [21 28 29 37]
 [22 29 30 38]
 [23 30 31 39]
 [25 31 33 40]
 [32 33 41 42]
 [34 43 35 44]
 [27 35 36 45]
 [28 36 37 46]
 [29 37 38 47]
 [30 38 39 48]
 [31 39 40 49]
 [33 40 42

Note that in addition to give how many elements we would like to have in each direction.
We also have to supply the _MPI-communicator_.
This is to specify how we would like the program to behave in parallel.
If we supply {py:data}`MPI.COMM_WORLD<mpi4py.MPI.COMM_WORLD>` we create a single mesh,
whose data is distributed over the number of processors we would like to use.
We can for instance run the program in  parallel on two processors by using `mpirun`, as:
``` bash
 mpirun -n 2 python3 t1.py
```
However, if we would like to create a separate mesh on each processor,
we can use {py:data}`MPI.COMM_SELF<mpi4py.MPI.COMM_SELF>`.
This is for instance  useful if we run a small problem, and would like to run it with multiple parameters.

--> 큰 문제를 다룰 때는 하나의 거대한 mesh에 대한 핸들링을 여러 MPI 프로세서에 쪼개어 처리하게 하고, 작은 문제에 대해 여러 케이스를 시험하고 싶으면 하나의 작은 mesh를 각 MPI 프로세서에 동일하게 담게 하고 각각의 케이스에 해당하는 문제를 풀게 하면 됨.

## Defining the finite element function space
 Once the mesh has been created, we can create the finite element function space $V$.
The finite element function space does not need to be the same as the one used to describe the mesh.
DOLFINx supports a wide range of arbitrary order finite element function spaces, see:
[Supported elements in DOLFINx](https://defelement.org/lists/implementations/basix.ufl.html)
for an extensive list.
To create a function space, we need to specify what mesh the space is defined on,
what element famil the space is based on, and the degree of the element.
These can for instance be defned through a tuple `("family", degree)`, as shown below

In [3]:
from dolfinx import fem

### 2. 사용할 (global) shape function 정하기 ; domain 전체에 대해 정의되는 함수라 global을 붙였음
V = fem.functionspace(domain, ("Lagrange", 1)) # FEM이 구해낼 근사 해 u가 속한 function space V의 order 설정. 걍 한 마디로 shape function order 설정
# V는 function space 객체를 의미

Further details about specification/customization of this tuple, see {py:class}`dolfinx.fem.ElementMetaData`.

##  Dirichlet boundary conditions
Next, we create a function that will hold the Dirichlet boundary data, and use interpolation to
fill it with the appropriate data.

In [3]:
### 3. 이따가 부여할 Dirichlet B.C에 대한 값을 만들기 위한 함수 미리 만들기(우리가 앞서 Dirichlet B.C를 spatially varying하게 설정했으니까, 단일 값이 아니라 공간 따라 변화하는 함수임)
uD = fem.Function(V) # functionspace V에 속하는 어떤 function 객체 uD 생성. uD는 boundary에서 정의되는 함수로 만들 거임. 따라서 Dirichlet BC를 만족하는 애로 만들거임
uD.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2) # uD = 1 + x^2 + 2y^2 ... 원래의 해석적 solution과 같음(당연함 ∵ boundary도 domain에 속하니까)

We now have the boundary data (and in this case the solution of the finite element problem)
represented in the discrete function space.
Next we would like to apply the boundary values to all degrees of freedom that are on the
boundary of the discrete domain.
We start by identifying the facets (line-segments) representing the outer boundary,
using {py:func}`dolfinx.mesh.exterior_facet_indices`.
We start by creating the facet to cell connectivity required to determine boundary facets by
calling {py:meth}`dolfinx.mesh.Topology.create_connectivity`.

In [ ]:
### 4. Dirichlet B.C를 부여할 boundary에 대해 정의하기
tdim = domain.topology.dim # domain이라는 mesh가 표현하는 topology의 차원 -> 즉 우리가 만든 mesh의 차원(2D)을 의미
# domain.topology : mesh가 포함하는 점(point), 점을 기반으로 한 선분(interval), 이 선분을 기반으로 한 면(face ... quadrilateral 등), 이 면을 기반으로 한 볼륨(cell ... tetrahedral 등)에 대한 정보.
# 이 모든 것들은 점을 linear하게 연결한다는 가정 위에 세워진 개념들.
# domain.geometry : mesh가 포함하는 형상 정의에 필요한 geometric points(nodes)에 대한 정보. 단, topology와는 다르게 level(점/선/면/볼륨)별로 정보를 관리하지 않음. level 개념이 없음. 오직 point(좌표)만 관리.
# 그리고 domain.geometry는 topology와 다르게 그 형상을 실제로 어떻게 표현했는지를 반영. 즉 어떤 형상을 곡선이나 곡면을 나타낼 수도 있도록 higher order로 nonlinear하게 표현했다면
# nonlinear에 쓰인 점의 정보까지 다 포함함. topology는 반면 linear 연결성만을 반영하기에, 같은 mesh에서의 geometry 정보의 점(point)의 개수가 topology 정보의 vertex 개수보다 많을 수 있음

fdim = tdim - 1 # 1D(2D 문제에서의 boundary는 1D를 의미. f는 facet, 즉 측면을 뜻함. 걍 바운더리 의미한다고 보면 됨)

domain.topology.create_connectivity(fdim, tdim) # domain이라는 mesh가 지닌 topology에 대하여 얘가 가지고 있는 모든 1D entity(edge)와 2D entity(face)를 서로 참조 가능하게 해줄거임.
# -> 이를 통해 mesh가 포함하는 tdim(2D) topological entity와 fdim(1D) topological entity 간 상호 포함(연결)관계가 정립된다(어떤 face entity가 어떤 edge entity들을 가지고(연결되어) 있는지).
# 이 작업이 있어야만 B.C를 boundary에 부여할 topological entity들을 specify할 수 있을 걸로 추정. 즉 이 명령어를 때려야만 바로 아랫줄의 exterior_facet_indices 함수를 쓸 수 있다.
# exterior_facet_indices 함수는 어떤 topology에 대해 자동으로 경계(boundary)에 있는 topological entity들을 식별할 수 있게 하는 함수니까 사전에 domain에 해당하는 topological entity들과 boundary에 해당하는 topological entity들 간 관계를 당연히 미리 알아야 함.
# ※ 참고 : mesh를 처음 생성하면(initialize), default로 최상위 topological entity(여기서는 2D 형상인 face)들과 최하위 topological entity(0D인 vertex) 간 connectivity를 지니게 됨.

boundary_facets = mesh.exterior_facet_indices(domain.topology) # domain이라는 mesh가 가진 최고차원 topology(domain.topology)의 한 단계 아래 차원을 지닌 topology(facet)의 entity(edge) 중 exterior(즉 boundary)에 위치한 edge entity들의 index만 추출(exterior_facet_indices)

# ※ 명심하셈. 여기서 다룬 건 모두 생성된 mesh가 포함하는 'topological entity(위상 요소)'에 관련된 작업임
# 즉 실제로 Dirichlet B.C 부여할 DOF index는 아직 선정 안 한 상태임(당연한거임 B.C는 원래 geometry/topology가 아니라 finite element가 가진 DOFs에다가 부여하는 거임).

In [10]:
boundary_facets # domain의 boundary에 해당하는 topology entity들 index

array([  0,   1,   3,   5,   9,  13,  17,  23,  27,  35,  39,  49,  53,
        65,  69,  83,  86,  87, 100, 101, 102, 113, 114, 123, 124, 131,
       132, 137, 138, 141, 142, 143], dtype=int32)

For the current problem, as we are using the first order Lagrange function space,
the degrees of freedom are located at the vertices of each cell, thus each facet contains two degrees of freedom.

To find the local indices of these degrees of freedom, we use {py:func}`dolfinx.fem.locate_dofs_topological`
which takes in the function space, the dimension of entities in the mesh we would like to identify and the local entities.
```{admonition} Local ordering of degrees of freedom and mesh vertices
Many people expect there to be a 1-1 correspondence between the mesh coordinates and the coordinates of the degrees of freedom.
However, this is only true in the case of `Lagrange` 1 elements on a first order mesh.
Therefore, in DOLFINx we use separate local numbering for the mesh coordinates and the dof coordinates.
To obtain the local dof coordinates we can use
{py:meth}`V.tabulate_dof_coordinates()<dolfinx.fem.FunctionSpace.tabulate_dof_coordinates>`,
while the ordering of the local vertices can be obtained by {py:attr}`mesh.geometry.x<dolfinx.mesh.Geometry.x>`.
```
With this data at hand, we can create the Dirichlet boundary condition

In [6]:
### 5. Dirichlet B.C 및 해당 B.C 부여할 DOF 정의
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
# Dirichlet B.C 부여할 DOFs 정의하는 명령어
# B.C 부여할 topology entity의 차원(fdim) 및 index(boundary_facets)와 이 index의 entity들에 대응되는 global shape function V의 DOFs를 뽑아내야 하기에 위와 같은 argument 필요.
bc = fem.dirichletbc(uD, boundary_dofs) # Dirichlet B.C 값(uD) 및 적용 DOF(boundary_dofs) 들어있는 객체 생성. 얘를 이따가 문제 정의 시에 input argument로 때려넣을거임.

In [12]:
boundary_dofs

array([ 0,  1,  2,  4,  6,  8, 11, 13, 17, 19, 24, 26, 32, 34, 41, 43, 44,
       51, 52, 53, 59, 60, 65, 66, 70, 71, 74, 75, 77, 78, 79, 80],
      dtype=int32)

## Defining the trial and test function

In mathematics, we distinguish between trial and test spaces $V$ and $\hat{V}$.
The only difference in the present problem is the boundary conditions.
In FEniCSx, we do not specify boundary conditions as part of the function space,
so it is sufficient to use a common space for the trial and test function.

We use the {py:mod}`Unified Form Language<ufl>` (UFL) to specify the varitional formulations.
See {cite}`fundamentals-ufl2014` for more details.

In [8]:
### 6. trial function u & test function v 정의
import ufl

u = ufl.TrialFunction(V) # 앞서 미리 정의한 function space에 속하는 trial function u 만들고
v = ufl.TestFunction(V) # 앞서 미리 정의한 function space에 속하는 test function v 만든다.

## Defining the source term
As the source term is constant over the domain, we use {py:class}`dolfinx.fem.Constant`

In [ ]:
### 7. Source term(righ hand side) 정의
from dolfinx import default_scalar_type

f = fem.Constant(domain, default_scalar_type(-6)) # 우리가 기선정한 exact solution u(x1, x2)에 따라 문제의 source term f의 값은 -6으로 상수로 고정.

```{admonition} Compilation speed-up
Instead of wrapping $-6$ in a {py:class}`dolfinx.fem.Constant`, we could simply define $f$ as `f=-6`.
However, if we would like to change this parameter later in the simulation,
we would have to redefine our variational formulation.
The {py:attr}`dolfinx.fem.Constant.value` allows us to update the value in $f$ by using `f.value=5`.
Additionally, by indicating that $f$ is a constant, we speed of compilation of the variational
formulations required for the created linear system.
```

## Defining the variational problem
As we now have defined all variables used to describe our variational problem, we can create the weak formulation

In [ ]:
### 8. Mathematical formulation : weak form 좌변(a(u, v)), 우변(L(v))을 구성하는 적분 term 정의
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx # ufl.dx는 x domain에 대한 integration을 의미
L = f * v * ufl.dx

Note that there is a very close correspondence between the Python syntax and the mathematical syntax
$\int_{\Omega} \nabla u \cdot \nabla v ~\mathrm{d} x$ and $\int_{\Omega}fv~\mathrm{d} x$.
The integration over the domain $\Omega$ is defined by using {py:func}`ufl.dx`, an integration
{py:class}`measure<ufl.Measure>` over all cells of the mesh.

This is the key strength of FEniCSx:
the formulas in the variational formulation translate directly to very similar Python code,
a feature that makes it easy to specify and solve complicated PDE problems.

## Expressing inner products
The inner product $\int_\Omega \nabla u \cdot \nabla v ~\mathrm{d} x$ can be expressed in various ways in UFL.
We have used the notation `ufl.dot(ufl.grad(u), ufl.grad(v))*ufl.dx`.
The {py:func}`dot<ufl.dot>` product in UFL computes the sum (contraction) over the last index
of the first factor and first index of the second factor.
In this case, both factors are tensors of rank one (vectors) and so the sum is just over
the single index of both $\nabla u$ and $\nabla v$.
To compute an inner product of matrices (with two indices),
one must use the function {py:func}`ufl.inner` instead of {py:func}`ufl.dot`.
For real-valued vectors, {py:func}`ufl.dot` and {py:func}`ufl.inner` are equivalent.

```{admonition} Complex numbers
In DOLFINx, one can solve complex number problems by using an installation of PETSc using complex numbers.
For variational formulations with complex numbers, one cannot use {py:func}`ufl.dot` to compute inner products.
One has to use {py:func}`ufl.inner`, with the test-function as the second input argument for {py:func}`ufl.inner`.
See [Running DOLFINx in complex mode](./complex_mode) for more information.
```


## Forming and solving the linear system

Having defined the finite element variational problem and boundary condition,
we can create our {py:class}`LinearProblem<dolfinx.fem.petsc.LinearProblem>` to solve the variational problem:
Find $u_h\in V$ such that $a(u_h, v)==L(v) \quad \forall v \in \hat{V}$.
We will use {py:mod}`PETSc<petsc4py.PETSc>` as our linear algebra backend, using a direct solver (LU-factorization).
See the [PETSc-documentation](https://petsc.org/main/docs/manual/ksp/?highlight=ksp#ksp-linear-system-solvers) of the method for more information.
PETSc is not a required dependency of DOLFINx, and therefore we explicitly import the DOLFINx wrapper for interfacing with PETSc.
To ensure that the options passed to the {py:class}`LinearProblem<dolfinx.fem.petsc.LinearProblem>`
is only used for the given KSP solver, we pass a **unique** option prefix as well.

In [ ]:
### 9. 앞서 기정의한 변수들(weak form 구성하는 적분 eqn / B.C)을 사용하여 문제 정의 / solver 설정 / solve
# 구하고자 하는 값(trial function u), test function(v), domain(mesh) 및 boundary 정보는 a, L, bc를 이루는 요소들에 다 녹아들어가있음
from dolfinx.fem.petsc import LinearProblem # petsc 패키지를 이용해서 linear system 문제를 풀건데 이미 dolfinx 내에 petsc가 wrapping 되어있으므로 해당 wrapper를 import한다.

problem = LinearProblem( # 선형 시스템(행렬 문제)
    a, # 좌변
    L, # 우변
    bcs=[bc], # 경계조건
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"}, # solver 설정
    petsc_options_prefix="Poisson", # 
)
uh = problem.solve() # h가 의미하는 것은 mesh size를 의미. 즉 fem으로 구한 해 u가 mesh size(h)에 dependent하다는 것을 의미.

# 사실 이렇게 fem.petsc 클래스 이용해서 petsc 패키지에 의존해서 문제 푸는 방법 말고도, fem.assemble_matrix 클래스 이용해서 문제 정의하는 법도 있음.
# 대신 이 방법으로 문제 풀 경우는 petsc 말고 다른 외부 라이브러리(scipy 같은)에 의존해서 문제를 풀어야 함(아니면 니가 직접 풀던가 ^오^).

In [20]:
uh

Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, quadrilateral, 1, gll_warped, unset, False, float64, []), (2,)), 1), Basix element (P, quadrilateral, 1, gll_warped, unset, False, float64, [])), 1)

Using {py:meth}`problem.solve()<dolfinx.fem.petsc.LinearProblem.solve>` we solve the linear system of equations and
return a {py:class}`Function<dolfinx.fem.Function>` containing the solution.

## Computing the error using solution uh from solved problem
Finally, we want to compute the error to check the accuracy of the solution.
We do this by comparing the finite element solution `u` with the exact solution.
First we interpolate the exact solution into a function space that contains it

In [ ]:
# 원래 문제의 exact solution인 1 + x_1^2 + 2*x_2^2를 선언(문제의 domain에 대해)
V2 = fem.functionspace(domain, ("Lagrange", 2)) # 2nd order shape function으로 함수 공간 설정
uex = fem.Function(V2, name="u_exact")
uex.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2)

We compute the error in two different ways.
First, we compute the $L^2$-norm of the error, defined by $E=\sqrt{\int_\Omega (u_D-u_h)^2\mathrm{d} x}$.
We use UFL to express the $L^2$-error, and use {py:func}`dolfinx.fem.assemble_scalar` to compute the scalar value.
In DOLFINx, {py:func}`assemble_scalar<dolfinx.fem.assemble_scalar>`
only assembles over the cells on the local process.
This means that if we use 2 processes to solve our problem,
we need to accumulate the local contributions to get the global error (on one or all processes).
We can do this with the {py:meth}`Comm.allreduce<mpi4py.MPI.Comm.allreduce>` function.

In [ ]:
# 이제 우리가 문제를 실제로 FEM으로 풀어서 구한 solution인 uh와 exact solution uex와의 error를 계산해보자.
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx) # 수식 정의(error 적분)
error_local = fem.assemble_scalar(L2_error) # element 하나에 대하여 정의된 수식(적분) 계산
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM)) # 전체 domain에 대하여 error_local 적분(sum)

Secondly, we compute the maximum error at any degree of freedom.
As the finite element function $u$ can be expressed as a linear combination of basis functions $\phi_j$,
spanning the space $V$: $ u = \sum_{j=1}^N U_j\phi_j.$
By writing {py:meth}`problem.solve()<dolfinx.fem.petsc.LinearProblem.sovle>`
we compute all the coefficients $U_1,\dots, U_N$.
These values are known as the _degrees of freedom_ (dofs).
We can access the degrees of freedom by accessing the underlying vector in `uh`.
However, as a second order function space has more dofs than a linear function space,
we cannot compare these arrays directly.  

--> 이게 뭔 소리냐면.. uh는 1차 shape function을 써서 구해진 애고, uex는 2차 함수니까 본질적으로 DOFs가 달라서(같은 mesh에 대해 연산했음에도 불구하고) uh의 계수로 구해진 애들, 즉 각 element의 node에 있는 u value와 uex에서의 계수 벡터가 가진 value들을 1대1로 비교가 어렵다는 얘기임(거의 DOFs 개수 차이가 2배 가까이 날 거기 때문). 따라서 uh와 uex 간 error 계산은 DOFs value가 아닌 각 전체 domain에 대해 각각의 함수로 적분한 값(L2 적분) 간 차이로 정의한 거임(바로 위의 코드와 같이).

As we already have interpolated the exact solution into the first order space when creating the boundary condition,
we can compare the maximum values at any degree of freedom of the approximation space.  

--> 이와 별개로 이제는 다음 코드와 같이 uh와 uD에서의 boundary value 간 차이를 볼 거임. uD는 B.C value 값 그 자체이기 때문에 진짜 값이라고 생각하면 됨. 반면 uh는 이 B.C를 만족하도록 구해진 함수이기에 '이론적으로는' 바운더리에서 uh - uD 값이 같을 거임. 허나 컴퓨터 계산이기 때문에 결국 machine precision만큼의 오차는 존재한다는 것을 보일 거임.

In [74]:
uex.x.array.shape

(289,)

In [ ]:
uD.x.array 
# uD가 가진 값들 보여줌
# uD가 8 x 8 elements로 짜여진 mesh에 대해 정의된 함수 공간 V에 속한 함수니까 node는 총 9 x 9 = 81개 존재
# boundary에서의 값만 신경쓰면 됨. 나머지 domain에 대한 값들은 사실상 uex에서의 그것과 같을 것임(같은 식을 썼기 때문)

array([1.      , 1.03125 , 1.015625, 1.046875, 1.125   , 1.140625,
       1.0625  , 1.09375 , 1.28125 , 1.296875, 1.1875  , 1.140625,
       1.171875, 1.5     , 1.515625, 1.34375 , 1.265625, 1.25    ,
       1.28125 , 1.78125 , 1.796875, 1.5625  , 1.421875, 1.375   ,
       1.390625, 1.421875, 2.125   , 2.140625, 1.84375 , 1.640625,
       1.53125 , 1.515625, 1.5625  , 1.59375 , 2.53125 , 2.546875,
       2.1875  , 1.921875, 1.75    , 1.671875, 1.6875  , 1.765625,
       1.796875, 3.      , 3.015625, 2.59375 , 2.265625, 2.03125 ,
       1.890625, 1.84375 , 1.890625, 2.      , 2.03125 , 3.0625  ,
       2.671875, 2.375   , 2.171875, 2.0625  , 2.046875, 2.125   ,
       3.140625, 2.78125 , 2.515625, 2.34375 , 2.265625, 2.28125 ,
       3.25    , 2.921875, 2.6875  , 2.546875, 2.5     , 3.390625,
       3.09375 , 2.890625, 2.78125 , 3.5625  , 3.296875, 3.125   ,
       3.765625, 3.53125 , 4.      ])

In [ ]:
uh.x.array # uh도 마찬가지로 8 x 8 elements로 짜여진 mesh에 대해 정의된 함수 공간 V에 속한 함수니까 node는 총 9 x 9 = 81개 존재

array([1.      , 1.03125 , 1.015625, 1.046875, 1.125   , 1.140625,
       1.0625  , 1.09375 , 1.28125 , 1.296875, 1.1875  , 1.140625,
       1.171875, 1.5     , 1.515625, 1.34375 , 1.265625, 1.25    ,
       1.28125 , 1.78125 , 1.796875, 1.5625  , 1.421875, 1.375   ,
       1.390625, 1.421875, 2.125   , 2.140625, 1.84375 , 1.640625,
       1.53125 , 1.515625, 1.5625  , 1.59375 , 2.53125 , 2.546875,
       2.1875  , 1.921875, 1.75    , 1.671875, 1.6875  , 1.765625,
       1.796875, 3.      , 3.015625, 2.59375 , 2.265625, 2.03125 ,
       1.890625, 1.84375 , 1.890625, 2.      , 2.03125 , 3.0625  ,
       2.671875, 2.375   , 2.171875, 2.0625  , 2.046875, 2.125   ,
       3.140625, 2.78125 , 2.515625, 2.34375 , 2.265625, 2.28125 ,
       3.25    , 2.921875, 2.6875  , 2.546875, 2.5     , 3.390625,
       3.09375 , 2.890625, 2.78125 , 3.5625  , 3.296875, 3.125   ,
       3.765625, 3.53125 , 4.      ])

In [72]:
error_max = numpy.max(numpy.abs(uD.x.array - uh.x.array))
if domain.comm.rank == 0:  # Only print the error on one process
    print(f"Error_L2 : {error_L2:.2e}") # exact solution uex와 fem으로 구한 solution uh 간 L2 에러(꽤 크게 나는 편임을 확인 가능)
    print(f"Error_max : {error_max:.2e}") # uh와 uD 간 에러(이론적으로는 0이나 컴퓨터 계산으로 인해 machine precision 보임)

Error_L2 : 8.24e-03
Error_max : 3.55e-15


## Plotting the mesh using pyvista
We will visualizing the mesh using [pyvista](https://docs.pyvista.org/), an interface to the VTK toolkit.
We start by converting the mesh to a format that can be used with {py:mod}`pyvista`.
To do this we use the function {py:func}`dolfinx.plot.vtk_mesh`.
It creates the data required to create a {py:class}`pyvista.UnstructuredGrid`.
You can print the current backend and change it with {py:func}`pyvista.set_jupyter_backend`.

In [75]:
import pyvista

print(pyvista.global_theme.jupyter_backend)

trame


In [ ]:
from dolfinx import plot

domain.topology.create_connectivity(tdim, tdim) # domain mesh가 가진 최상위 레벨 geometry entity들 간 연결 정보(어디가 어디랑 인접해있고 이런 거)를 생성
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim) # vtk_mesh 함수를 통해서 domain mesh의 최상위 레벨 geometry 요소에 대한 vtk 포맷 데이터를 생성
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry) # 위에서 만든 vtk 포맷 데이터를 pyvista의 UnstructuredGrid에 넣어서 vtk 포맷의 형상 데이터 생성

There are several backends that can be used with pyvista, and they have different benefits and drawbacks.
See the [pyvista documentation](https://docs.pyvista.org/user-guide/jupyter/index.html#state-of-3d-interactive-jupyterlab-plotting)
for more information and installation details.

We can now use the {py:class}`pyvista.Plotter` to visualize the mesh. We visualize it by showing it in 2D and warped in 3D.
In the jupyter notebook environment, we use the default setting of `pyvista.OFF_SCREEN=False`,
which will render plots directly in the notebook.

In [ ]:
# 문제에서 정의된 domain에 대한 mesh visualization ... 여기서 말하는 mesh란, geometry 개념의 mesh(우리가 앞서 만든 변수 'domain')임. 따라서 우리가 solution field(u)를 구할 DOFs와는 무관.
# vtk(pyvista)의 경우 geometric 사각형 셀(quads)을 triangle 2개로 분할하여 렌더링하는 경우가 거의 기본값이기 때문에 이미지 보면 삼각형으로 잘려있을거임.
pyvista.OFF_SCREEN = True
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")

## Plotting a function using pyvista
We want to plot the solution `uh`.
As the function space used to defined the mesh is decoupled from the representation of the mesh,
we create a mesh based on the dof coordinates for the function space `V`.
We use {py:func}`dolfinx.plot.vtk_mesh` with the function space as input to create a mesh with
mesh geometry based on the dof coordinates.

In [ ]:
# fem으로 구한 uh에 대한 plot(마침내!)을 하기 위해 우선 uh가 정의된 함수공간인 V에 대해 vtk_mesh 함수를 갈겨서 
# V가 정의된 domain(mesh)이 어떤 토폴로지, element type, geometry(vertices들의 좌표)를 가지고 있는지 먼저 정보 생성
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)

Next, we create the {py:class}`pyvista.UnstructuredGrid` and add the dof-values to the mesh.

In [ ]:
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry) # V가 정의된 domain에 대한 정보를 pyvista에다가 넘겨서 unstructured grid 객체 생성
u_grid.point_data["u"] = uh.x.array.real # uh가 가지고 있는 dofs 값들을 u_grid 객체에 넘김
u_grid.set_active_scalars("u") # 왜있는지모르겠는데..뭐 이걸 해야 plot을 할 수 있나부지..
u_plotter = pyvista.Plotter() # plot 객체 생성
u_plotter.add_mesh(u_grid, show_edges=True) # 그 plot 객체에다가 mesh로써 u_grid 넣어버림
u_plotter.view_xy() # plot에다가 xy 축 넣는거인듯
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    figure2 = u_plotter.screenshot("fundamentals_uh.png")

We can also warp the mesh by scalar to make use of the 3D plotting.

In [ ]:
# 3D로 플롯
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()
else:
    figure3 = plotter2.screenshot("fundamentals_uh_3D.png")

## External post-processing
For post-processing outside the python code, it is suggested to save the solution to file using either
{py:class}`dolfinx.io.VTXWriter` or {py:class}`dolfinx.io.XDMFFile` and using [Paraview](https://www.paraview.org/).
This is especially suggested for 3D visualization.

In [ ]:
from dolfinx import io
from pathlib import Path

results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "fundamentals"
with io.VTXWriter(domain.comm, filename.with_suffix(".bp"), [uh]) as vtx:
    vtx.write(0.0)
with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh)

```{bibliography}
   :filter: cited
   :labelprefix:
   :keyprefix: fundamentals-
```